In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [2]:
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

test_ratio = 0.8

train_data = []
test_data = []

for type_name in set(dataset['type']):
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]
    
    test_size = int(len(curr_type_data) * test_ratio)
    
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

def format_data(sample):
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])
    
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

원본 데이터의 type 분포:
mrc_question_with_1_to_4_negative: 296
paraphrased_question: 196
synthetic_question: 497
mrc_question: 491
no_answer: 404

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
mrc_question_with_1_to_4_negative: 60
paraphrased_question: 40
synthetic_question: 100
mrc_question: 99
no_answer: 81

테스트 데이터의 type 분포:
mrc_question_with_1_to_4_negative: 236
paraphrased_question: 156
synthetic_question: 397
mrc_question: 392
no_answer: 323


In [3]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: “우버(UBER)는 사람들이 이동하는 수단을 발전시켰습니다. 승객과 기사를 실시간으로 연결해 승객에게는 편리함을, 기사에게는 더 많은 효율성과 수익을 가져다 주었습니다. 2010년 6월 미국 샌프란시스코에서 처음 서비스를 시작한 뒤 현재 세계 140여개 도시에 진출했습니다. 우버는 앞으로도 계속 사람과 도시를 가깝게 이어줄 것입니다.”샌프란시스코 우버 본사에서 최근 만난 나이리 후다지안 우버 글로벌커뮤니케이션 부문장은 이렇게 강조했다. 우버는 일종의 ‘차량 예약 서비스’를 제공하는 글로벌 정보기술(IT) 기업이다. 스마트폰 애플리케이션(앱·응용프로그램·사진)을 통해 승객과 차량을 연결해 주는 사업을 하고 있다.승객을 일반택시와 연결해 주는 ‘우버택시’, 일반인이 자신의 차량으로 운송 서비스를 할 수 있도록 도와주는 ‘우버엑스’, 일종의 고급 콜택시인 ‘우버블랙’ 등의 서비스를 갖췄다. 한국에서는 지난해 8월 우버코리아가 설립돼 우버블랙 사업을 펼치고 있다.후다지안 부문장은 “우리는 더 나은 교통 서비스 위해 플랫폼을 제공하는 회사”라

In [4]:
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [5]:
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1504 [00:00<?, ? examples/s]

In [6]:
train_dataset[0]

{'messages': [{'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: “박근혜 대통령의 방중을 통해 북핵문제에 대해 한·미·중 삼각협력체제가 형성될 수 있는 기반이 마련된 것은 외교적 업적이 분명하다.” 서진영 고려대 명예교수(71·사진)는 30일 “박 대통령의 방중이 상당히 만족할 만한 효과를 거뒀다”고 평가하며 이같이 말했다. 서 교수는 중국 정치와 미·중관계 등을 연구해온 원로 중국전문가다. 김영삼 정부에서 대통령 자문 정책기획위원장을, 이명박 정부에서 한·중 전문가 공동연구위원회 한국 측 위원장을 역임했다. 서 교수는 한·미·중 정상이 연쇄적으로 회동하면서 북한 핵문제에 대해 일치된 목소리를 낸 것에 가장 큰 의미를 부여했다. 그는 “이번 한·중 정상회담에서 ‘북한 핵을 절대 용납할 수 없다’는 원칙을 ‘한·중 미래비전 공동선언’에 담지는 못했지만 박 대통령이 기자회견에서 직접 언급했다는 점에서 중국이 간접적으로 인정했다고 볼 수 있다”며 “과거보다 중국이 북핵문제에 대해 훨씬 전향적인 자세를 보인 것”이라고 말했다.특히 중국 측이 박 대통령에게 보인 환대를 주목했다. 서 교수는 “시진핑 주석은 박 대통령과

In [7]:
model_id = "Qwen/Qwen2-7B-Instruct" 

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: “박근혜 대통령의 방중을 통해 북핵문제에 대해 한·미·중 삼각협력체제가 형성될 수 있는 기반이 마련된 것은 외교적 업적이 분명하다.” 서진영 고려대 명예교수(71·사진)는 30일 “박 대통령의 방중이 상당히 만족할 만한 효과를 거뒀다”고 평가하며 이같이 말했다. 서 교수는 중국 정치와 미·중관계 등을 연구해온 원로 중국전문가다. 김영삼 정부에서 대통령 자문 정책기획위원장을, 이명박 정부에서 한·중 전문가 공동연구위원회 한국 측 위원장을 역임했다. 서 교수는 한·미·중 정상이 연쇄적으로 회동하면서 북한 핵문제에 대해 일치된 목소리를 낸 것에 가장 큰 의미를 부여했다. 그는 “이번 한·중 정상회담에서 ‘북한 핵을 절대 용납할 수 없다’는 원칙을 ‘한·중 미래비전 공동선언’에 담지는 못했지만 박 대통령이 기자회견에서 직접 언급했다는 점에서 중국이 간접적으로 인정했다고 볼 수 있다”며 “과거보다 중국이 북핵문제에 대해 훨씬 전향적인 자세를 보인 것”이라고 말했다.특히 중국 측이 박 대통령에게 보인 환대를 주목했다. 서 교수는 “시진핑 주석은 박 대통령과 7, 8시간을 같이 보냈고 부인인 

In [9]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [10]:
args = SFTConfig(
    output_dir="qwen2-7b-rag-ko",
    num_train_epochs=3, 
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [11]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        labels = [-100] * len(input_ids)
        
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):

                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

In [12]:
max_seq_length=8192

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 2998])
어텐션 마스크 형태: torch.Size([1, 2998])
레이블 형태: torch.Size([1, 2998])


In [13]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 138520, 19391, 143604, 129264, 130650, 382, 13146, 48431, 20401, 66790, 29326, 131193, 17877, 125686, 125548, 139713, 624, 16, 13, 138520, 53680, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 16186, 139713, 624, 17, 13, 85322, 77226, 98801, 19391, 130768, 130213, 17877, 143604, 16186, 125476, 34395, 53900, 21329, 95577, 139713, 624, 18, 13, 138520, 19391, 128605, 143603, 12802, 85322, 77226, 98801, 19391, 130671, 32290, 85322, 77226, 98801, 126377, 330, 33883, 64795, 138520, 93, 19391, 128605, 130213, 12802, 136673, 1189, 5140, 45881, 34395, 143604, 16186, 139713, 624, 19, 13, 143604, 47836, 53618, 142976, 139236, 18411, 142616, 82190, 53435, 40853, 129549, 53435, 125068, 17877, 140174, 128836, 32290, 5140, 240, 97, 19391, 36330, 250, 125746, 16560, 23084, 126402, 83634, 17380, 94613, 139236, 84621, 47324, 18411, 129624, 20487, 139713, 13, 95617, 18411, 129901, 26698

In [14]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [15]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [16]:
trainer.train()

trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.555300
20,0.519200
30,0.468300
40,0.396100
50,0.456200
60,0.394200
70,0.481000
80,0.454400
90,0.388600
100,0.358800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

In [17]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [18]:
print(prompt_lst[42])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: “업사이클링은 새로운 디자인 패러다임으로 자리매김할 것입니다.”이태용 한국디자인진흥원장(사진)은 업사이클링에 대해 “투자 대비 부가가치가 높고 환경문제를 해결하기 위한 촉매제 역할을 할 것”이라며 이같이 말했다. 이 원장은 “꾸준한 연구와 시범사업 등을 통해 업사이클링 산업을 육성해 나가겠다”고 덧붙였다.현재 국내 업체들은 업사이클링 제품의 대량 생산과 유통망 확보에 큰 어려움을 겪고 있다. 이 원장은 “업사이클링 분야에서는 1인 기업이 많아 대량 생산에 한계가 있다”며 “오프라인 매장을 두지 못하고 온라인을 통해서만 판매를 하는 업체들이 많다”고 설명했다.이 같은 문제점을 해결하기 위해 한국디자인진흥원은 작년부터 ‘업사이클 디자인 사업’을 진행하고 있다. 지난 10월 열린 ‘디자인코리아 2013’에서 업사이클디자인관을 마련해 홍보를 도왔다. 전문인력 양성을 위한 세미나도 개최했다. 국내 인프라 구축을 위해 다양한 연구 활동도 하고 있다. 이 원장은 “업사이클링 산업이 발전하면 새로운 시장과 일자리가 창출될 수 있을 것”이라며 “마케팅 부문 등에서 실질적인 도움을 주는 데 주력하고

In [19]:
print(label_lst[42])


기업에서 오픈프라이즈를 활용할 수 있는 분야는 주로 마케팅과 신제품 홍보입니다. 오픈프라이즈는 소비자에게 무료로 제품을 나눠주는 경품추첨 서비스를 제공하여, 기업이 신제품을 짧은 기간 내에 다수의 소비자에게 노출시킬 수 있는 효과적인 마케팅 수단으로 활용될 수 있습니다. 이를 통해 기업은 현물 투자 방식으로 비용을 절감하면서도 소비자 만족도를 높일 수 있습니다. 또한, 소비자들이 제품 후기를 작성하거나 설문에 응답하는 등의 활동을 통해 추가적인 마케팅 데이터를 수집할 수 있습니다 [[ref3]].<|im_end|>



In [20]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [22]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [23]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [24]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [25]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
청춘밴드는 사연 신청자의 부모님을 위해 특별한 선물을 생각했습니다. 사연 신청자는 부모님을 위해 특별한 선물을 하고 싶어 했고, 그 부모님은 비닐 공장을 운영하는 사람입니다. '청춘밴드'는 부모님의 취향을 저격하는 즉석 노래들을 선보이며, 그들의 마음을 사로잡았습니다. 또한, '청춘밴드'는 부모님의 취향을 저격하는 즉석 노래들을 선보이며, 그들의 마음을 사로잡았습니다. 이에 '청춘밴드'는 남다른 추억을 선물해주기 위해 '리마인드 웨딩 이벤트'를 진행했습니다. 이 이벤트는 부모님에게 특별한 추억을 선물하는 것으로, '청춘밴드'는 이 일로 시청자들에게 감동을 선사했습니다 [[ref5]].
    label:

'청춘밴드'는 사연 신청자의 부모님을 위해 특별한 선물로 리마인드 웨딩 이벤트를 준비했습니다. 사연 신청자의 부모님은 한평생 일하느라 청춘을 잊고 지내셨고, 흔한 결혼사진 한 장 없이 40여 년의 세월을 함께 걸어온 부부였습니다. '청춘밴드'는 이 부부에게 남다른 추억을 선물해주기 위해 리마인드 웨딩 이벤트를 진행하며, 부모님의 취향을 저격하는 즉석 노래들을 선보여 마음을 사로잡았습니다. 이 이벤트는 부모님에게 특별한 추억을 만들어주고, 가족들과 함께 웃음과 눈물을 나누는 감동적인 시간이 되었습니다 [[ref5]].<|im_end|>

--------------------------------------------------
    response:
GS 신임 임원 과정의 마무리 행사가 지난 1일부터 6박7일간 진행되었으며, 이는 6일에 열린 신임 임원 만찬으로 마무리되었습니다 [[ref2]].
    label:

GS 신임 임원 과정의 마무리 행사는 지난 6일에 제주시 엘리시안 제주리조트에서 열렸습니다. 이 행사는 1일부터 6박 7일간 진행된 GS 신임 임원 과정의 마지막 행사로, 신임 임원 20여명 등 30여명이 참석했습니다 [[ref2]].<|im_end|>

-------------------------------------